In [1]:
## this example adapted from a linear regression example at:
## https://www.geeksforgeeks.org/linear-regression-using-pytorch/

In [2]:
import torch
from torch.autograd import Variable
from scipy.spatial import distance as dist

In [3]:
x_data = Variable(torch.Tensor([[0.0, 0.0], [1.0, 0.0], [2.0, 0.0]])) ## "trace"
y_data = Variable(torch.Tensor([[0.0, 1.0], [1.0, 1.0], [2.0, 1.0]])) ## "reference"

In [4]:
x_data

tensor([[0., 0.],
        [1., 0.],
        [2., 0.]])

In [5]:
y_data

tensor([[0., 1.],
        [1., 1.],
        [2., 1.]])

In [29]:
class LinearTransform(torch.nn.Module):
 
    def __init__(self):
        super(LinearTransform, self).__init__()
        self.transform = torch.nn.Linear(2, 2, bias=True)  # two in and two out
        self.transform.weight = torch.nn.Parameter( torch.tensor([[1.0,0.0],[0.0,1.0]], requires_grad=True))
        self.transform.bias = torch.nn.Parameter( torch.tensor([0.0,0.0], requires_grad=True))
        print 'init', self.transform.weight.data.numpy()
        print self.transform.bias
 
    def forward(self, x):
        y_pred = self.transform(x)
        return y_pred


def custom_loss(tra_verts, ref_verts):
    sum_dist = torch.tensor(0.0)
    for i, v in enumerate(tra_verts):
        rv = ref_verts[i]
        current_dist = ((v[0] - rv[0]) ** 2 + (v[1] - rv[1]) ** 2) ** 1/2
        sum_dist.add_(current_dist)
        
    return torch.tensor(sum_dist, requires_grad=True)

In [30]:
# our model
model = LinearTransform()

init [[1. 0.]
 [0. 1.]]
Parameter containing:
tensor([0., 0.], requires_grad=True)


In [31]:
# define a loss and optimizer

## loss
## TODO: this should perhaps be a different, to take into account the 
## "cost" of the transformation as measured by: (1) the "area swept out by the stroke
## upon being moved into place plus (2) the "residual_area" between the trace and reference
# criterion = torch.nn.MSELoss(size_average=True) 

## optimizier
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [32]:
num_train_steps = 5000
for i,epoch in enumerate(range(num_train_steps)):
 
    # Forward pass: Compute predicted y by passing 
    # x to the model
    pred_y = model(x_data)
 
    # Compute and print loss
    loss = custom_loss(pred_y, y_data)
    
    # Zero gradients, perform a backward pass, 
    # and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%1000==0:
        print('epoch {}, loss {}'.format(epoch, loss.data[0]))


/Users/zixianchai/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch 0, loss 1.5
epoch 1000, loss 1.35931319534e-08
epoch 2000, loss 3.55449003564e-12
epoch 3000, loss 3.55449003564e-12
epoch 4000, loss 3.55449003564e-12


In [33]:
## this will "freeze" the weights of the model so we can apply it
model.eval()

LinearTransform(
  (transform): Linear(in_features=2, out_features=2, bias=True)
)

In [34]:
## uncomment the below to get the matrix in numpy form
weight_matrix = model.transform.weight.data.numpy()
print 'weight matrix'
print weight_matrix
print ' '
bias_matrix = model.transform.bias.data.numpy()
print 'bias matrix'
print bias_matrix

weight matrix
[[1.0000000e+00 0.0000000e+00]
 [1.4896989e-06 1.0000000e+00]]
 
bias matrix
[0.         0.99999756]


In [35]:
## does it handle a new point properly? (i.e., move it up by 1 unit along the y-axis?)
new_var = Variable(torch.Tensor([[3.0,0.0]]))
pred_y = model(new_var)
print("predict (after training)", model(new_var).data)

('predict (after training)', tensor([[3.0000, 1.0000]]))


In [ ]:
# try scaling
x_data = Variable(torch.Tensor([[0.0, 0.0], [1.0, 0.0], [2.0, 0.0]])) ## "trace"
y_data = Variable(torch.Tensor([[-1.0, 0.0], [1.0, 0.0], [3.0, 0.0]])) ## "reference"

num_train_steps = 10000
for i,epoch in enumerate(range(num_train_steps)):
 
    # Forward pass: Compute predicted y by passing 
    # x to the model
    pred_y = model(x_data)
 
    # Compute and print loss
    loss = criterion(pred_y, y_data)
 
    # Zero gradients, perform a backward pass, 
    # and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%1000==0:
        print('epoch {}, loss {}'.format(epoch, loss.data[0]))

In [ ]:
## does it handle a new point properly? (i.e., move it up by 1 unit along the y-axis?)
new_var = Variable(torch.Tensor([[3.0,0.0], [5.0, 0.0], [7.0, 0.0]]))
pred_y = model(new_var)
print("predict (after training)", model(new_var).data)